# NER 모델 생성 테스트

Hugging Face Transformers 라이브러리와 KoELECTRA 모델을 사용

In [1]:
# 설치 필요한 라이브러리
# !pip install torch transformers scikit-learn seqeval
# !pip install accelerate

# 라벨 정의

## 핵심 라벨 (KCD분류의 뼈대)

- 신체부위 (BODY) : 질병/상해 발생 위치
    ex) 머리, 허리, 좌측 무릎, L4-5번 척추

- 주진단 (DIS-MAIN) : 의학적으로 확정된 병명
    ex) 골절, 염좌, 위염, 디스크, 뇌진탕

- 증상 (SYMPTOM) : 진단명이 나오기 전, 환자가 느끼는 주관적 고통(R코드 예측에 중요)
    ex) 호흡곤란, 통증, 불편함, 어지러움


## 맥락 라벨 (상해/질병 구분용)

- 사고원인 (CAUSE) : 상해 예측의 핵심. 외력에 의한 것인지 파악.
    ex) 교통사고, 낙상, 미끄러짐, 접촉사고, 부딪힘

- 행동/상황 (ACT) : 상해 발생 상황 파악. 사고 당시 무엇을 하고 있었는가? (산재 여부나 상해 기전 파악)
    ex) 운전 중, 작업 중, 보행 중, 축구 하다가

- 시점/기간 (TIME) : 금성인지 만성인지 구분.
    ex) 갑자기, 예전부터, 만성적인, 3일전부터


## 심화 라벨 (세부 코드 결정용)

- 치료/수술 (TREATMENT) : 수술 여부는 중증도(코드의 앞자리)를 암시함.
    ex) 봉합(열상), 핀삽입(골절), 물리치료, 입원

- 방향/측면 (SIDE) : 좌/우/양측 (최근 KCD는 방향에 따라 코드가 갈림)
    ex) 좌측, 우측, 양측

- 과거력 (DIS-HIST) : 이번 사고/청구와 직접 관련 없는 과거의 병력이나 기저 질환.
    ex) (허리)수술 이력, (고혈압)약 복용 중, 예전에 다친 적 있음, 지병

- 부정어 (NEG) : 부정어가 포함된 경우. 골절 아님을 골절로 인식하면 안됨
    ex) (골절)없음, 아님, (이상)소견 없음


# 라벨링 하기

1차 라벨링은 Doccano와 ChatGPT를 사용한다.
라벨링 JSON 포맷은 아래와 같다.

```json
{
    "text": "이순신은 서울에서 거북선을 만들었다.",
    "entities": [
        {
            "text": "이순신",
            "label": "PER",
            "start_offset": 0,
            "end_offset": 3
        },
        {
            "text": "서울",
            "label": "CITY",
            "start_offset": 5,
            "end_offset": 7
        },
        {
            "text": "거북선",
            "label": "ARTIFACT",
            "start_offset": 10,
            "end_offset": 13
        }
    ]
}
```

In [2]:
import torch
import numpy as np
from transformers import (
    AutoTokenizer, 
    AutoModelForTokenClassification, 
    TrainingArguments, 
    Trainer,
    DataCollatorForTokenClassification
)
from torch.utils.data import Dataset

# ---------------------------------------------------------
# 1. 설정 및 태그 정의 (Configuration)
# ---------------------------------------------------------
MODEL_NAME = "monologg/koelectra-base-v3-discriminator" # 한국어 성능이 우수한 모델
MAX_LEN = 128

# 우리가 정의한 태그 리스트 (BIO Scheme)
# B: 시작, I: 중간, O: 관련없음
TAGS = [
    "O",
    "B-DIS-MAIN", "I-DIS-MAIN",   # 주진단 (예: 용종, 골절)
    "B-DIS-HIST", "I-DIS-HIST",   # 과거력 (예: 위식도 역류성 질환)
    "B-SYMPTOM", "I-SYMPTOM",     # 증상 (예: 호흡곤란, 통증)
    "B-BODY", "I-BODY",           # 신체부위 (예: 대장, 손)
    "B-TREATMENT", "I-TREATMENT", # 치료/수술 (예: 용종점막절제술)
    "B-CAUSE", "I-CAUSE"          # 사고원인 (예: 낙하물)
]

# 태그 <-> ID 매핑 생성
label2id = {tag: i for i, tag in enumerate(TAGS)}
id2label = {i: tag for i, tag in enumerate(TAGS)}



# ---------------------------------------------------------
# 2. 데이터셋 클래스 정의 (Dataset)
# ---------------------------------------------------------
class MedicalNERDataset(Dataset):
    def __init__(self, data, tokenizer, label2id, max_len):
        self.data = data
        self.tokenizer = tokenizer
        self.label2id = label2id
        self.max_len = max_len

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        item = self.data[idx]
        text = item['text']
        # 실제 학습시는 레이블링 툴(Doccano 등)에서 뽑은 정답 리스트가 들어와야 함
        # 여기서는 데모를 위해 텍스트만 처리하거나, 더미 레이블을 매핑하는 로직이 필요
        # *주의*: 실제 학습 데이터는 문장과 함께 [O, O, B-DIS, ...] 형태의 라벨 리스트가 있어야 함
        
        # 토크나이징 (Subword 단위 분리)
        encoding = self.tokenizer(
            text,
            padding='max_length',
            truncation=True,
            max_length=self.max_len,
            return_tensors='pt'
        )
        
        # 데모용: 실제 정답 라벨이 있다고 가정하고 텐서 변환
        # (실제 프로젝트에선 여기서 subword align 로직이 복잡하게 들어감)
        labels = item.get('labels', [0] * self.max_len) 
        labels = torch.tensor(labels, dtype=torch.long)

        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': labels
        }

# ---------------------------------------------------------
# [데이터 생성 도우미 함수]
# 문장과 태깅할 단어 리스트를 주면, 모델용 포맷(labels)으로 변환해줍니다.
# ---------------------------------------------------------
def make_training_data(tokenizer, text, entities, max_len):
    """
    text: 원본 문장
    entities: {"단어": "태그명"} 형태의 딕셔너리
    max_len: 최대 길이
    """
    # 1. 토크나이징 (Offset Mapping 포함: 토큰이 원문의 몇 번째 글자인지 위치 정보 반환)
    tokenized = tokenizer(
        text, 
        padding='max_length', 
        truncation=True, 
        max_length=max_len, 
        return_offsets_mapping=True
    )
    
    # 2. 라벨 리스트 초기화 (모두 'O'(=0)으로 시작)
    labels = [label2id["O"]] * max_len
    offsets = tokenized['offset_mapping']
    
    # 3. 각 Entity 위치 찾아서 라벨링 (BIO 태깅)
    for word, tag_name in entities.items():
        # 문장에서 단어의 시작 위치 찾기
        start_char = text.find(word)
        if start_char == -1: continue # 단어가 없으면 스킵
        end_char = start_char + len(word)
        
        # 각 토큰이 이 단어 범위 안에 있는지 확인
        # 예: '위식도'(0~3) -> 토큰 '위'(0~1), '식도'(1~3)
        found_start = False
        for idx, (offset_start, offset_end) in enumerate(offsets):
            if offset_start == 0 and offset_end == 0: continue # 특수토큰 스킵
            
            # 토큰이 단어 범위 내에 완전히 포함되면
            if offset_start >= start_char and offset_end <= end_char:
                if not found_start:
                    # 첫 토큰은 B-태그
                    labels[idx] = label2id[f"B-{tag_name}"]
                    found_start = True
                else:
                    # 나머지 토큰은 I-태그
                    labels[idx] = label2id[f"I-{tag_name}"]
                    
    return {
        'text': text,
        'labels': labels
    }

# ---------------------------------------------------------
# [사용 예시] - 이렇게 데이터를 추가하세요!
# ---------------------------------------------------------

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

training_data = []

# 데이터 1
data1 = make_training_data(
    tokenizer,
    text="위식도 역류성 질환으로 통원 치료를 받았다.",
    entities={
        "위식도 역류성 질환": "DIS-MAIN", # 주진단
        "통원 치료": "TREATMENT"          # 치료
    },
    max_len=MAX_LEN
)
training_data.append(data1)

# 데이터 2
data2 = make_training_data(
    tokenizer,
    text="호흡곤란으로 응급실 내원하여 폐렴 진단받음",
    entities={
        "호흡곤란": "SYMPTOM",
        "폐렴": "DIS-MAIN"
    },
    max_len=MAX_LEN
)
training_data.append(data2)

# ... 계속 추가 ...
{
  "text": "눈 통증으로 진료",
  "entities": {
    "눈": "BODY",
    "통증": "SYMPTOM",
    "진료": "TREATMENT"
  }
}

# 데이터셋 인스턴스 생성
print(f"생성된 데이터 개수: {len(training_data)}")
train_dataset = MedicalNERDataset(training_data, tokenizer, label2id, MAX_LEN)


# ---------------------------------------------------------
# 4. 모델 초기화 및 학습 설정 (Training Setup)
# ---------------------------------------------------------
model = AutoModelForTokenClassification.from_pretrained(
    MODEL_NAME,
    num_labels=len(TAGS),
    id2label=id2label,
    label2id=label2id
)

training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,              # 에포크 수 (NER은 금방 과적합되므로 적게)
    per_device_train_batch_size=8,
    learning_rate=5e-5,              # 미세조정용 낮은 학습률
    logging_dir='./logs',
    logging_steps=10,
    save_strategy="no",              # 데모용이라 저장 안 함
    use_cpu=False                    # GPU 있으면 False
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    data_collator=DataCollatorForTokenClassification(tokenizer)
)

# 학습 시작 (더미 데이터라 금방 끝남)
print(">>> 학습 시작 (Dummy Data)...")
trainer.train()
print(">>> 학습 완료!")

# ---------------------------------------------------------
# 5. 추론 및 후처리 로직 (Inference & Post-processing)
# ---------------------------------------------------------
def predict_ner(text, model, tokenizer):
    """
    입력 텍스트에서 Entity를 추출하여 보기 좋게 반환하는 함수
    """
    model.eval()
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    # 입력 처리
    inputs = tokenizer(text, return_tensors="pt", truncation=True, max_length=MAX_LEN)
    inputs = {k: v.to(device) for k, v in inputs.items()}

    # 예측
    with torch.no_grad():
        outputs = model(**inputs)
    
    logits = outputs.logits
    predictions = torch.argmax(logits, dim=2)
    
    # 결과 디코딩
    tokens = tokenizer.convert_ids_to_tokens(inputs['input_ids'][0])
    predicted_labels = [id2label[p.item()] for p in predictions[0]]

    # 결과 정리 (Subword 병합 및 Entity 그룹화)
    entities = []
    current_entity = {"word": "", "label": None}
    
    for token, label in zip(tokens, predicted_labels):
        if token in ["[CLS]", "[SEP]", "[PAD]"]:
            continue
            
        # Subword 처리 ('##'으로 시작하는 토큰 병합)
        clean_token = token.replace("##", "")
        
        if label.startswith("B-"):
            # 이전 Entity 저장
            if current_entity["word"]:
                entities.append(current_entity)
            # 새 Entity 시작
            current_entity = {"word": clean_token, "label": label[2:]} # "B-" 제거
            
        elif label.startswith("I-") and current_entity["label"] == label[2:]:
            # 현재 Entity에 이어붙이기
            current_entity["word"] += clean_token
            
        else: # "O" 태그이거나 라벨이 끊긴 경우
            if current_entity["word"]:
                entities.append(current_entity)
                current_entity = {"word": "", "label": None}
                
    if current_entity["word"]:
        entities.append(current_entity)

    return entities

# ---------------------------------------------------------
# 6. 실제 테스트 (Demo)
# ---------------------------------------------------------
# *참고*: 학습 데이터가 더미(0)라서 결과는 엉망이겠지만, 로직 흐름은 확인 가능
test_text = "위식도 역류성 질환으로 통원 치료를 받던 중 호흡곤란으로 대장내시경 검사를 받았는데 용종이 커져서 영향을 받은 것으로 용종점막절제술을 받게 된 것입니다."

print("\n>>> [테스트 문장]:", test_text)
result = predict_ner(test_text, model, tokenizer)

print("\n>>> [NER 추출 결과]")
for entity in result:
    print(f"Entity: {entity['word']}  |  Label: {entity['label']}")

# ---------------------------------------------------------
# 7. KCD 코드 매핑용 데이터 구조화 (Tip)
# ---------------------------------------------------------
final_structure = {
    "주진단(MAIN)": [],
    "부위(BODY)": [],
    "증상(SYMPTOM)": [],
    "과거력(HIST)": [],
    "수술(TREATMENT)": []
}

for item in result:
    tag = item['label']
    word = item['word']
    
    if tag == "DIS-MAIN": final_structure["주진단(MAIN)"].append(word)
    elif tag == "BODY": final_structure["부위(BODY)"].append(word)
    elif tag == "SYMPTOM": final_structure["증상(SYMPTOM)"].append(word)
    elif tag == "DIS-HIST": final_structure["과거력(HIST)"].append(word)
    elif tag == "TREATMENT": final_structure["수술(TREATMENT)"].append(word)

print("\n>>> [KCD 매핑을 위한 최종 구조]")
print(final_structure)

생성된 데이터 개수: 2


Some weights of ElectraForTokenClassification were not initialized from the model checkpoint at monologg/koelectra-base-v3-discriminator and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


>>> 학습 시작 (Dummy Data)...


/Users/passion1014/project/axlrator/ml/venv/lib/python3.11/site-packages/torch/utils/data/dataloader.py:692: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, device pinned memory won't be used.
  warnings.warn(warn_msg)


Step,Training Loss


>>> 학습 완료!

>>> [테스트 문장]: 위식도 역류성 질환으로 통원 치료를 받던 중 호흡곤란으로 대장내시경 검사를 받았는데 용종이 커져서 영향을 받은 것으로 용종점막절제술을 받게 된 것입니다.

>>> [NER 추출 결과]
Entity: 를  |  Label: CAUSE
Entity: 으로  |  Label: CAUSE
Entity: 를  |  Label: CAUSE
Entity: 서  |  Label: CAUSE
Entity: 을  |  Label: CAUSE
Entity: 으로  |  Label: CAUSE
Entity: 을  |  Label: CAUSE
Entity: .  |  Label: CAUSE

>>> [KCD 매핑을 위한 최종 구조]
{'주진단(MAIN)': [], '부위(BODY)': [], '증상(SYMPTOM)': [], '과거력(HIST)': [], '수술(TREATMENT)': []}


In [3]:
from kiwipiepy import Kiwi

kiwi = Kiwi()

def preprocess_text(text):
    """
    형태소 분석기를 사용해 명사와 조사 사이를 강제로 띄어줍니다.
    예: "위식도질환은" -> "위식도 질환 은"
    """
    results = kiwi.analyze(text)
    tokens = []
    for token, pos, _, _ in results[0][0]:
        # 필요하다면 여기서 불용어 제거나 특정 품사만 남길 수도 있음
        tokens.append(token)
    return " ".join(tokens) # 띄어쓰기로 연결

# ---------------------------------------------------
# 비교 테스트
# ---------------------------------------------------
raw_text = "위식도역류질환은 통증이 심하다"

# 1. 그냥 BERT 토크나이저
print(f"원본 그대로: {tokenizer.tokenize(raw_text)}")
# 결과 예시: ['위', '##식', '##도', '##역', '##류', '##질환은', ...] (조사가 붙을 수 있음)

# 2. 형태소 분석 후 BERT 토크나이저
preprocessed = preprocess_text(raw_text)
print(f"형태소 처리: {preprocessed}")
print(f"처리 후 토큰: {tokenizer.tokenize(preprocessed)}")
# 결과 예시: "위식도 역류 질환 은 통증 이 심하다"
# 토큰: ['위', '##식', '##도', '역류', '질환', '은', '통증', '이', ...] (조사가 확실히 떨어짐!)

Quantization is not supported for ArchType::neon. Fall back to non-quantized model.


원본 그대로: ['위', '##식', '##도', '##역', '##류', '##질', '##환', '##은', '통증', '##이', '심하', '##다']
형태소 처리: 위 식도 역류 질환 은 통증 이 심하 다
처리 후 토큰: ['위', '식도', '역류', '질환', '은', '통증', '이', '심하', '다']


# 테스트 참고 사항

In [4]:
from kiwipiepy import Kiwi

kiwi = Kiwi()
text = "위내시경 검사를 받았습니다."

# analyze 함수로 분석
result = kiwi.analyze(text, top_n=1)

for token, tag, start, _len in result[0][0]:
    print(f"{token} ({tag})")

# [출력 결과]
# 위내시경 (NNG) -> 일반 명사
# 검사 (NNG)
# 를 (JKO) -> 목적격 조사 (BERT 분리 대상!)
# 받 (VV) -> 동사
# 었습니다 (EF) -> 어미
# . (SF)

위 (NNG)
내시경 (NNG)
검사 (NNG)
를 (JKO)
받 (VV-R)
었 (EP)
습니다 (EF)
. (SF)


Quantization is not supported for ArchType::neon. Fall back to non-quantized model.


In [5]:
def clean_text_with_kiwi(text, kiwi_model):
    """
    1. 의료 용어 사전을 기반으로 형태소를 분석한다.
    2. 조사(Josa)를 떼어내기 위해 공백을 삽입한다.
    3. 정제된 텍스트를 반환한다.
    """
    # 형태소 분석 결과 가져오기
    tokens = kiwi_model.tokenize(text)
    
    # 형태소(form)만 공백으로 연결
    # 예: [위식도, 역류, 질환, 은] -> "위식도 역류 질환 은"
    cleaned_text = " ".join([t.form for t in tokens])
    
    return cleaned_text

# 사용
raw_text = "위식도역류질환은 통증이심함"
processed_text = clean_text_with_kiwi(raw_text, kiwi)

# 이 processed_text를 BERT Tokenizer에 넣으면 성능 대폭 향상!
print(processed_text) 
# "위식도역류질환 은 통증 이 심하 ㅁ"

위 식도 역류 질환 은 통증 이 심하 ᆷ
